In [1]:
%%capture
!pip install transformers datasets

     |████████████████████████████████| 2.5MB 27.3MB/s 
     |████████████████████████████████| 245kB 49.4MB/s 
     |████████████████████████████████| 901kB 50.8MB/s 
     |████████████████████████████████| 3.3MB 51.6MB/s 
     |████████████████████████████████| 245kB 54.4MB/s 
     |████████████████████████████████| 122kB 47.5MB/s 


In [2]:
import itertools as it
import logging
import math
import re
import sys
import warnings
from dataclasses import dataclass, field
from typing import Optional

import datasets
import torch

import soundfile as sf
from transformers import HfArgumentParser, Wav2Vec2ForCTC, Wav2Vec2Processor

In [3]:
kenlm_model_path = "" #@param {type:"string"}

In [4]:
#@title Beam options
beam_size = 200 #@param {type:"integer"}
beam_threshold = 25.0 #@param {type:"number"}

In [5]:
import math
word_score = 1.0
# weight of <unk>nown token
unk_weight = -math.inf
# weight of <sil>ence token
sil_weight = 0.0

# Set up kenlm

In [6]:
%cd /content

/content


In [7]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 14047, done.
remote: Counting objects: 100% (360/360), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 14047 (delta 107), reused 121 (delta 55), pack-reused 13687
Receiving objects: 100% (14047/14047), 5.76 MiB | 19.78 MiB/s, done.
Resolving deltas: 100% (7987/7987), done.


In [8]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [9]:
%%capture
%cd /content/kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4

In [10]:
%%capture
%cd /content/kenlm
!python setup.py install
%cd /tmp

In [11]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/content/kenlm/build/bin/"

# Set up flashlight python bindings

In [12]:
%cd /content

/content


In [13]:
!git clone https://github.com/flashlight/flashlight

Cloning into 'flashlight'...
remote: Enumerating objects: 17649, done.
remote: Counting objects: 100% (1523/1523), done.
remote: Compressing objects: 100% (714/714), done.
remote: Total 17649 (delta 827), reused 1332 (delta 765), pack-reused 16126
Receiving objects: 100% (17649/17649), 14.23 MiB | 27.81 MiB/s, done.
Resolving deltas: 100% (12298/12298), done.


In [14]:
%%capture
!apt install -q libfftw3-dev

In [15]:
%%capture
!pip install packaging

In [16]:
%cd flashlight/bindings/python

/content/flashlight/bindings/python


In [18]:
%env USE_MKL=0
%env KENLM_ROOT=/content/kenlm
!python setup.py install

env: USE_MKL=0
env: KENLM_ROOT=/content/kenlm
running install
running bdist_egg
running egg_info
writing flashlight.egg-info/PKG-INFO
writing dependency_links to flashlight.egg-info/dependency_links.txt
writing top-level names to flashlight.egg-info/top_level.txt
package init file 'flashlight/__init__.py' not found (or not a regular file)
package init file 'flashlight/lib/audio/__init__.py' not found (or not a regular file)
package init file 'flashlight/lib/sequence/__init__.py' not found (or not a regular file)
package init file 'flashlight/lib/text/__init__.py' not found (or not a regular file)
reading manifest template 'MANIFEST.in'
writing manifest file 'flashlight.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
-- -rdynamic supported.
-- CUDA found (library: /usr/local/cuda/lib64/libcudart_static.a;-lpthread;dl;/usr/lib/x86_64-linux-gnu/librt.so include: /usr/local/cuda/include)
-- CUDA architectu